In [ ]:
from multiprocessing import cpu_count
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
import vk
import urllib.request
import numpy as np
from PIL import Image
import re
import json
%matplotlib inline
nltk.download('stopwords')

In [ ]:
token = "9cad73ddf0088749120e2376c2dfcd1a8d89a26b42a3e0e39b3a47833247525e63f4a18b27f41b46d409d"

lemmatizer = nltk.stem.WordNetLemmatizer()

cores = cpu_count()

In [ ]:
def get_groups():
    session = vk.Session(token)
    api = vk.API(session)
    try:
        json_response = api.groups.search(q="a", type="page", count="1000", v="5.131", sort="6")
        return json_response['items']
    except Exception as ex:
        print(ex)

def get_mean_reposts(posts):
    mean_reposts = 0
    for i in range(len(posts)):
        mean_reposts += posts[i]['reposts']['count']
    mean_reposts = mean_reposts / len(posts)
    return mean_reposts

def get_posts(group_id, count, offset):
    session = vk.Session(token)
    api = vk.API(session)
    try:
        json_response = api.wall.get(owner_id=-group_id, count=count, offset=offset, v="5.131")
        posts = json_response['items']
        mean_reposts = get_mean_reposts(posts)
        for i in range(len(posts)):
            posts[i]['virality'] = (posts[i]['reposts']['count'] / mean_reposts) * 1000
        return posts
    except Exception as ex:
        print(ex)

In [ ]:
groups = get_groups()
groups = groups[:100]

In [ ]:
posts = []
for i in range(len(groups)):
    posts = posts + get_posts(groups[i]['id'], 100, 5)

Предобработка

In [ ]:
dfc = pd.DataFrame(posts).to_dict('list')

In [ ]:
def first_photo_average_colour(attachments):
    try:
        if attachments != None and attachments == attachments:
            photos = list(filter(lambda x: x['type']=='photo', attachments))
            if any(photos):
                url = photos[0]['photo']['sizes'][-1]['url']
                urllib.request.urlretrieve(url, "temp.png")
                img = np.asarray(Image.open('temp.png'))
                average_colour_row = np.average(img, axis=0)
                average_colour = np.average(average_colour_row, axis=0)
                return list(average_colour)
        return [0, 0, 0]
    except:
        return [0, 0, 0]

colours = []
for i in range(len(dfc['attachments'])):
    colours = colours + [first_photo_average_colour(dfc['attachments'][i])]

In [ ]:
def get_word_count(text):
    text = text.lower()
    text = re.sub("[^A-Za-zА-Яа-я]", " ", text)
    text = word_tokenize(text)
    return(len(text))

word_counts = []
for i in range(len(dfc['text'])):
    word_counts = word_counts + [get_word_count(dfc['text'][i])]

In [ ]:
def get_photo_count(attachments):
    if attachments != None and attachments == attachments:
        photos = list(filter(lambda x: x['type']=='photo', attachments))
        return(len(photos))
    return 0

picture_counts = []
for i in range(len(dfc['attachments'])):
    picture_counts = picture_counts + [get_photo_count(dfc['attachments'][i])]

In [ ]:
def get_video_count(attachments):
    if attachments != None and attachments == attachments:
        videos = list(filter(lambda x: x['type']=='video', attachments))
        return(len(videos))
    return 0

video_counts = []
for i in range(len(dfc['attachments'])):
    video_counts = video_counts + [get_video_count(dfc['attachments'][i])]

In [ ]:
def get_audio_count(attachments):
    if attachments != None and attachments == attachments:
        audios = list(filter(lambda x: x['type']=='audio', attachments))
        return(len(audios))
    return 0

audio_counts = []
for i in range(len(dfc['attachments'])):
    audio_counts = audio_counts + [get_audio_count(dfc['attachments'][i])]

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub("[^A-Za-zА-Яа-я]", " ", text)
    text = word_tokenize(text)
    text = [word for word in text if word not in set(nltk.corpus.stopwords.words("russian"))]
    text = [lemmatizer.lemmatize(word) for word in text]
    return text

texts = []
for i in range(len(dfc['text'])):
    texts = texts + [preprocess_text(dfc['text'][i])]

Запись в файл

In [ ]:
data = {}
data['text'] = texts
data['photo'] = picture_counts
data['video'] = video_counts
data['audio'] = audio_counts
data['colour'] = colours
data['words'] = list(map(lambda x: len(x), texts))
data['likes'] = list(map(lambda x: x['count'], dfc['likes']))
data['reposts'] = list(map(lambda x: x['count'], dfc['reposts']))
data['views'] = list(map(lambda x: x['count'], dfc['views']))
data['comments'] = list(map(lambda x: x['count'], dfc['comments']))
data['virality'] = dfc['virality']

with open('data.json', 'w') as file:
    json.dump(data, file)